<a href="https://colab.research.google.com/github/Mafaz03/Model/blob/main/XOR_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Import Dependencies

In [1]:
import torch
from torch import nn
import torch.utils.data as data_utils

import numpy as np
import pandas as pd

#2. Create Dataset

In [8]:
# For Visualization purpose
XOR = pd.DataFrame( { 0 : [0 , 0 , 1 , 1] , 1 : [0 , 1 , 0 , 1] , "Target" : [0 , 1 , 1 , 0] } )
XOR

,0,1,Target
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,0


In [3]:
## Actual Dataset to be Used

X_train = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y_train = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# Create a DataLoader
train_dataset = data_utils.TensorDataset(X_train, y_train)
train_loader = data_utils.DataLoader(train_dataset, batch_size=1, shuffle=True)

In [4]:
next(iter(train_loader))

[tensor([[1., 0.]]), tensor([[1.]])]

#3. Model Creation

In [20]:
class XOR_model(nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = nn.Linear(2 , 10)
    self.fc2 = nn.Linear(10 , 1)

  def forward(self , x):

    x = torch.relu(self.fc1(x))
    x = torch.sigmoid(self.fc2(x))

    return x

model = XOR_model()
model.parameters()

<generator object Module.parameters at 0x7f96e2cf5c40>

#4. Training Loop

In [32]:
%%time

optimizer = torch.optim.Adam(params = model.parameters() , lr = 0.01)
loss_fn = nn.BCELoss()

epochs = 100
model.train()

for epoch in range(epochs+1):
  total_loss = 0

  for data, target in train_loader:

    optimizer.zero_grad()

    y_pred = model(data)
    loss = loss_fn(y_pred, target)

    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  if epoch % 10 == 0:
    print(f"Epoch {epoch} || Training loss : {total_loss}")

Epoch 0 || Training loss : 0.1008217204362154
Epoch 10 || Training loss : 0.05507797468453646
Epoch 20 || Training loss : 0.03637756314128637
Epoch 30 || Training loss : 0.025259461253881454
Epoch 40 || Training loss : 0.018626802368089557
Epoch 50 || Training loss : 0.01419415371492505
Epoch 60 || Training loss : 0.011173223960213363
Epoch 70 || Training loss : 0.00898349576164037
Epoch 80 || Training loss : 0.0074260468245483935
Epoch 90 || Training loss : 0.006241014751140028
Epoch 100 || Training loss : 0.005298510193824768
CPU times: user 251 ms, sys: 1.56 ms, total: 253 ms
Wall time: 254 ms


#5. Testing

In [31]:
acc = 0
y = [0 , 1 , 1 , 0]
with torch.no_grad():
    test_input = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
    predictions = model(test_input)
    rounded_predictions = torch.round(predictions)
    print("Predictions:", rounded_predictions.reshape(1,-1))

    for elements in range(len(rounded_predictions)):
      if int(rounded_predictions[elements]) == y[elements]:
        acc += 1
print(f"Accuracy : {acc / len(rounded_predictions) *100}%")


Predictions: tensor([[0., 1., 1., 0.]])
Accuracy : 100.0%
